In [ ]:
import time
import requests
import pandas as pd

from datetime import datetime
from tqdm.notebook import tqdm
from user_agent import generate_user_agent


#### Collect Data

In [ ]:
user_agents = generate_user_agent()
api_key = "RGAPI-6b662dab-1426-4c62-a5e5-b01e9eb2ed3e"
header = {
          "User-Agent" : user_agents,
          "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
          "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
          "Origin": "https://developer.riotgames.com",
          "X-Riot-Token": api_key
}

In [ ]:
def league_v4(tier):
    queue = "RANKED_SOLO_5x5"

    if tier == "challenger":
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    elif tier == "grandmaster":
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    else:
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"

    return requests.get(url, headers=header)

In [ ]:
req = league_v4("master")

if req.status_code == 200:
    data = req.json()["entries"]

    summonerId = [item["summonerId"] for item in data]
else:
    print(req.status_code)

len(summonerId)

#### 소환사 범위 정하기

In [ ]:
resize_summonerId = summonerId[0:100]
len(resize_summonerId)

In [ ]:
def summoner_v4(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}"

    return requests.get(url, headers=header)

In [ ]:
puuId = []
for _summonerId in tqdm(resize_summonerId):
    req = summoner_v4(_summonerId)

    if req.status_code == 200:
        data = req.json()

        puuId.append(data["puuid"])
    else:
        print(req.status_code)
    time.sleep(1.3)
len(puuId)

In [ ]:
def match_v5(puuid, startTime, endTime, type, start, count):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&endTime={endTime}&type={type}&start={start}&count={count}"
    
    return requests.get(url, headers=header)

In [ ]:
startTime = int(datetime(2023, 2, 1, 0, 0, 0).timestamp())
endTime = int(datetime(2023, 12, 31, 23, 59, 59).timestamp())
type = 'ranked'
startTime, endTime

In [ ]:
matchId = []

for _puuId in tqdm(puuId):
    req = match_v5(_puuId, startTime, endTime, type, 0, 100)

    if req.status_code == 200:
        data = req.json()
        matchId.append(data)
    else:
        print(req.status_code)
    time.sleep(1.3)
len(matchId), print(matchId)

In [ ]:
def matchData(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"

    return requests.get(url, headers=header)

In [ ]:
match_data = []
for matchIds in tqdm(matchId):
    for _matchId in matchIds:
        req = matchData(_matchId)
        if req.status_code == 200:
            data = req.json()
            match_data.append(data)
        else:
            print(req.status_code)
            break
        time.sleep(1.3)
len(match_data)

In [ ]:
match_data